# 1. Main code.

In [1]:
%load_ext Cython

In [2]:
%%cython --annotate
from math import log 

cpdef ProcessRT(t,int i,int l,int nn1):
    """ This function recursively extends an input tuple of integers, which represents 
    a set of sets (more precisily, a formal context), by adding to it an integer i (a new set).
    The length of a terminal tuple is constrained by l, while nn1 should be passed as 2**n-1. 
    The function checks whether the new tuple forms a unique closure system and returns the dictionary of the first elements of
    processed tuples as keys and values 1 for clousre system with T1 propery and 0 otherwise."""
    
    count={}
    t=t[:]
    cdef int P
    cdef int lent=len(t)

    
    #checking whether the context is reducible
    for j in range(lent-1,-1,-1):
        P=i
        #print("t[j],i",t[j],i,t[j]&i,)
        tji=t[j]&i
        if tji in t[0:j]:
            return count
            
        if tji==t[j]:
            #print("t[j],i",t[j],i,t[j]&i,)
            for h in range(j+1,lent):
                if t[j]&t[h]==t[j]:
                    P=P&t[h]
                    if P==t[j]:                        
                        return count
    t.append(i)

    #checking T1 property
    if Check(t,nn1):
        count[(lent+1,tuple(t))]=1
    else:
        count[(lent+1,tuple(t))]=0

    #recursive call
    cdef unsigned long long r
    #print(l)
    if l!=0:
        r=0
        for k in range(i+1,nn1):
            #print("Process(t,k,l-1,tuples,nn1)")
            #print("t",t,"k",k)
            ucount=ProcessRT(t,k,l-1,nn1)
            for key in ucount:
                if key in count:
                    count[key]+=ucount[key]
                else:
                    count[key]=ucount[key]
            #if r<0: print("r",r)
        return count
    else: 
        #print(tuples)
        #tuples.append(tuple(t))
       
        
        print("Branch process",t[0])
        return count

    
def isClosed(t,g):
    """Checks whether g is closed in t."""
    cont=[]
    for h in t:
        if g&h==g:
            cont.append(h)
    if len(cont)>0:
        res=cont[0]
        for h in cont:
            res=h&res
        return res==g
    else:
        return g==0

    
def Check(t,nn1):
    """Checks whether T1 is fulfilled for t."""
    n=int(log(nn1+1,2))
    members=[]
    members.extend([2**i for i in range(n)])
    for m in members:
        if not isClosed(t,m):
            return False
    return True

def merge(r):
    """This is a value collector for the resulting list of dictionaries
    """
    merged={}
    for dic in r:
        for key in dic:
            if key in merged:
                merged[key]+=dic[key]
            else:
                merged[key]=dic[key]
    return merged


def merge2(r):
    """This is a value collector for the resulting list of dictionaries
    with non-zero values.
    """
    merged={}
    for dic in r:
        for key in dic:
            if key in merged and dic[key]==1:
                merged[key]+=dic[key]
            else:
                if dic[key]==1:
                    merged[key]=dic[key]
    return merged

# 2. Counting inequivalent A334254 for n=2, 3, 4, 5.

In [3]:
%%time



from multiprocessing import Pool
from multiprocessing import cpu_count


n=5

#computing A334254 for n=5 by levels
if __name__ == "__main__":
    pool = Pool(cpu_count())
    
    nn1=2**n-1

    lt=[]
    lk=[]

    count={}

    
    for t in range(nn1):
 
        for i in range(t+1,nn1):
            lt.append([t])
            lk.append(i)
            
    ln_2=[nn1-1]* len(lt)
    lnn1=[nn1]* len(lt)
           
    #print(lk)   
     
    print(list(zip(lt,lk,ln_2,lnn1)))

    
    #parallel execution of ProcessRT function
    res5 = pool.starmap_async(ProcessRT, zip(lt,lk,ln_2,lnn1))
    print(res5.get())# print the list of resulting dictionaries
    print(sum(merge(res5.get()).values()))#the number of closure systems w.r.t. T1

    pool.close()  # 'TERM'
    pool.join()   # 'KILL'

[([0], 1, 30, 31), ([0], 2, 30, 31), ([0], 3, 30, 31), ([0], 4, 30, 31), ([0], 5, 30, 31), ([0], 6, 30, 31), ([0], 7, 30, 31), ([0], 8, 30, 31), ([0], 9, 30, 31), ([0], 10, 30, 31), ([0], 11, 30, 31), ([0], 12, 30, 31), ([0], 13, 30, 31), ([0], 14, 30, 31), ([0], 15, 30, 31), ([0], 16, 30, 31), ([0], 17, 30, 31), ([0], 18, 30, 31), ([0], 19, 30, 31), ([0], 20, 30, 31), ([0], 21, 30, 31), ([0], 22, 30, 31), ([0], 23, 30, 31), ([0], 24, 30, 31), ([0], 25, 30, 31), ([0], 26, 30, 31), ([0], 27, 30, 31), ([0], 28, 30, 31), ([0], 29, 30, 31), ([0], 30, 30, 31), ([1], 2, 30, 31), ([1], 3, 30, 31), ([1], 4, 30, 31), ([1], 5, 30, 31), ([1], 6, 30, 31), ([1], 7, 30, 31), ([1], 8, 30, 31), ([1], 9, 30, 31), ([1], 10, 30, 31), ([1], 11, 30, 31), ([1], 12, 30, 31), ([1], 13, 30, 31), ([1], 14, 30, 31), ([1], 15, 30, 31), ([1], 16, 30, 31), ([1], 17, 30, 31), ([1], 18, 30, 31), ([1], 19, 30, 31), ([1], 20, 30, 31), ([1], 21, 30, 31), ([1], 22, 30, 31), ([1], 23, 30, 31), ([1], 24, 30, 31), ([1], 25,

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



CPU times: user 5.42 s, sys: 1.11 s, total: 6.52 s
Wall time: 11.4 s


In [4]:
%%time



from multiprocessing import Pool
from multiprocessing import cpu_count


n=4

#computing A334254 for n=4 by levels
if __name__ == "__main__":
    pool = Pool(cpu_count())
    
    nn1=2**n-1

    lt=[]
    lk=[]

    count={}

    
    for t in range(nn1):
 
        for i in range(t+1,nn1):
            lt.append([t])
            lk.append(i)
            
    ln_2=[nn1-1]* len(lt)
    lnn1=[nn1]* len(lt)
           
    #print(lk)   
     
    print(list(zip(lt,lk,ln_2,lnn1)))

    
    #parallel execution of ProcessRT function
    res4 = pool.starmap_async(ProcessRT, zip(lt,lk,ln_2,lnn1))
    print(res4.get())# print the list of resulting dictionaries
    print(sum(merge(res4.get()).values()))#the number of closure systems w.r.t. T1

    pool.close()  # 'TERM'
    pool.join()   # 'KILL'

[([0], 1, 14, 15), ([0], 2, 14, 15), ([0], 3, 14, 15), ([0], 4, 14, 15), ([0], 5, 14, 15), ([0], 6, 14, 15), ([0], 7, 14, 15), ([0], 8, 14, 15), ([0], 9, 14, 15), ([0], 10, 14, 15), ([0], 11, 14, 15), ([0], 12, 14, 15), ([0], 13, 14, 15), ([0], 14, 14, 15), ([1], 2, 14, 15), ([1], 3, 14, 15), ([1], 4, 14, 15), ([1], 5, 14, 15), ([1], 6, 14, 15), ([1], 7, 14, 15), ([1], 8, 14, 15), ([1], 9, 14, 15), ([1], 10, 14, 15), ([1], 11, 14, 15), ([1], 12, 14, 15), ([1], 13, 14, 15), ([1], 14, 14, 15), ([2], 3, 14, 15), ([2], 4, 14, 15), ([2], 5, 14, 15), ([2], 6, 14, 15), ([2], 7, 14, 15), ([2], 8, 14, 15), ([2], 9, 14, 15), ([2], 10, 14, 15), ([2], 11, 14, 15), ([2], 12, 14, 15), ([2], 13, 14, 15), ([2], 14, 14, 15), ([3], 4, 14, 15), ([3], 5, 14, 15), ([3], 6, 14, 15), ([3], 7, 14, 15), ([3], 8, 14, 15), ([3], 9, 14, 15), ([3], 10, 14, 15), ([3], 11, 14, 15), ([3], 12, 14, 15), ([3], 13, 14, 15), ([3], 14, 14, 15), ([4], 5, 14, 15), ([4], 6, 14, 15), ([4], 7, 14, 15), ([4], 8, 14, 15), ([4], 9

In [5]:
%%time



from multiprocessing import Pool
from multiprocessing import cpu_count


n=3

#computing A334254 for n=3 by levels
if __name__ == "__main__":
    pool = Pool(cpu_count())
    
    nn1=2**n-1

    lt=[]
    lk=[]

    count={}

    
    for t in range(nn1):
 
        for i in range(t+1,nn1):
            lt.append([t])
            lk.append(i)
            
    ln_2=[nn1-1]* len(lt)
    lnn1=[nn1]* len(lt)
           
    #print(lk)   
     
    print(list(zip(lt,lk,ln_2,lnn1)))

    
    #parallel execution of ProcessRT function
    res3 = pool.starmap_async(ProcessRT, zip(lt,lk,ln_2,lnn1))
    print(res3.get())# print the list of resulting dictionaries
    print(sum(merge(res3.get()).values()))#the number of closure systems w.r.t. T1

    pool.close()  # 'TERM'
    pool.join()   # 'KILL'

[([0], 1, 6, 7), ([0], 2, 6, 7), ([0], 3, 6, 7), ([0], 4, 6, 7), ([0], 5, 6, 7), ([0], 6, 6, 7), ([1], 2, 6, 7), ([1], 3, 6, 7), ([1], 4, 6, 7), ([1], 5, 6, 7), ([1], 6, 6, 7), ([2], 3, 6, 7), ([2], 4, 6, 7), ([2], 5, 6, 7), ([2], 6, 6, 7), ([3], 4, 6, 7), ([3], 5, 6, 7), ([3], 6, 6, 7), ([4], 5, 6, 7), ([4], 6, 6, 7), ([5], 6, 6, 7)]
[{(2, (0, 1)): 0, (3, (0, 1, 3)): 0, (3, (0, 1, 5)): 0}, {(2, (0, 2)): 0, (3, (0, 2, 3)): 0, (3, (0, 2, 6)): 0}, {(2, (0, 3)): 0, (3, (0, 3, 5)): 0, (3, (0, 3, 6)): 0}, {(2, (0, 4)): 0, (3, (0, 4, 5)): 0, (3, (0, 4, 6)): 0}, {(2, (0, 5)): 0, (3, (0, 5, 6)): 0}, {(2, (0, 6)): 0}, {(2, (1, 2)): 0, (3, (1, 2, 3)): 0, (4, (1, 2, 3, 4)): 1, (3, (1, 2, 4)): 1, (4, (1, 2, 4, 5)): 1, (4, (1, 2, 4, 6)): 1, (3, (1, 2, 5)): 0, (4, (1, 2, 5, 6)): 1, (3, (1, 2, 6)): 0}, {(2, (1, 3)): 0, (3, (1, 3, 4)): 0, (4, (1, 3, 4, 6)): 1, (3, (1, 3, 6)): 0}, {(2, (1, 4)): 0, (3, (1, 4, 5)): 0, (3, (1, 4, 6)): 0}, {(2, (1, 5)): 0, (3, (1, 5, 6)): 0}, {(2, (1, 6)): 0}, {(2, (2, 3))

In [6]:
%%time



from multiprocessing import Pool
from multiprocessing import cpu_count


n=2

#computing A334254 for n=3 by levels
if __name__ == "__main__":
    pool = Pool(cpu_count())
    
    nn1=2**n-1

    lt=[]
    lk=[]

    count={}

    
    for t in range(nn1):
 
        for i in range(t+1,nn1):
            lt.append([t])
            lk.append(i)
            
    ln_2=[nn1-1]* len(lt)
    lnn1=[nn1]* len(lt)
           
    #print(lk)   
     
    print(list(zip(lt,lk,ln_2,lnn1)))

    
    #parallel execution of ProcessRT function
    res2 = pool.starmap_async(ProcessRT, zip(lt,lk,ln_2,lnn1))
    print(res2.get())# print the list of resulting dictionaries
    print(sum(merge(res2.get()).values()))#the number of closure systems w.r.t. T1

    pool.close()  # 'TERM'
    pool.join()   # 'KILL'

[([0], 1, 2, 3), ([0], 2, 2, 3), ([1], 2, 2, 3)]
[{(2, (0, 1)): 0}, {(2, (0, 2)): 0}, {(2, (1, 2)): 1}]
1
CPU times: user 82.6 ms, sys: 481 ms, total: 563 ms
Wall time: 927 ms


# 2. Inequivalent closure systems (formal contexts) with respect to set permutations.

### Cases from n=2 to 5 are given. For n=0,1, a(0)=1 (empty set) and a(1)=2 ({1} and {{},1}).

In [7]:
def PermBit(bit,perm):
    out=0
    for i in range(len(perm)):
        if (bit>>i) & 1==1:
            out=out | (1<<perm[i])
    return out

def PermTuple(t,perm):
    ret=[]
    for bit in t:
        ret.append(PermBit(bit,perm))
    return tuple(ret)


def isLectLess(a,b):
    #is A lectically smaller than B?
    for i in range(len(a)):
        if a[i]==b[i]:
            pass
        elif a[i]<b[i]:
            return True
        else:
            return False
    return True
        
        
    

In [8]:
isLectLess((1,2,3),(1,2,3))

True

In [9]:
isLectLess((1,2,3),(1,3,4))

True

In [10]:
isLectLess((1,3,4),(1,2,3))

False

In [11]:
from itertools import permutations

In [12]:
n=3
Perms=list(permutations(range(n)))

In [13]:
d3=merge2(res3.get())

In [14]:
T3=[]
for key in d3.keys():
    T3.append(key[1])
    

In [15]:
T3

[(1, 2, 3, 4),
 (1, 2, 4),
 (1, 2, 4, 5),
 (1, 2, 4, 6),
 (1, 2, 5, 6),
 (1, 3, 4, 6),
 (2, 3, 4, 5),
 (3, 5, 6)]

In [16]:
PermTuple(T3[0],Perms[1])

(1, 4, 5, 2)

In [17]:
Perms[0]


(0, 1, 2)

In [18]:
ineqT3=[]


for t in T3:
    mode=True
    for p in Perms:
        pt=sorted(PermTuple(t,p))
        if isLectLess(t,pt):
            pass
        else:
            mode=False
            break
            
    if mode: 
        ineqT3.append(t)
            
            

In [19]:
ineqT3

[(1, 2, 3, 4), (1, 2, 4), (1, 2, 5, 6), (3, 5, 6)]

In [20]:
n=4

Perms=list(permutations(range(n)))

d4=merge2(res4.get())


T4=[]
for key in d4.keys():
    T4.append(key[1])


ineqT4=[]
for t in T4:
    mode=True
    for p in Perms:
        pt=sorted(PermTuple(t,p))
        if isLectLess(t,pt):
            pass
        else:
            mode=False
            break
            
            
    if mode: 
        ineqT4.append(t)
              

In [21]:
ineqT4

[(1, 2, 3, 4, 7, 8),
 (1, 2, 3, 4, 8),
 (1, 2, 3, 4, 8, 12),
 (1, 2, 3, 4, 11, 12),
 (1, 2, 4, 7, 8),
 (1, 2, 4, 7, 8, 11),
 (1, 2, 4, 8),
 (1, 2, 4, 9, 10),
 (1, 2, 4, 9, 10, 11),
 (1, 2, 4, 9, 10, 11, 12),
 (1, 2, 4, 9, 10, 12),
 (1, 2, 4, 9, 10, 13),
 (1, 2, 4, 9, 10, 13, 14),
 (1, 2, 4, 9, 11, 14),
 (1, 2, 4, 9, 14),
 (1, 2, 4, 11, 13, 14),
 (1, 2, 5, 10, 12),
 (1, 2, 5, 10, 12, 13),
 (1, 2, 5, 10, 13, 14),
 (1, 2, 7, 11, 12),
 (1, 3, 6, 7, 10, 12),
 (1, 3, 6, 10, 12),
 (1, 3, 6, 10, 12, 14),
 (1, 3, 6, 11, 12),
 (1, 3, 6, 11, 12, 14),
 (1, 6, 7, 10, 11, 12),
 (1, 6, 7, 10, 12),
 (1, 6, 7, 10, 13),
 (1, 6, 10, 12),
 (1, 6, 10, 12, 14),
 (1, 6, 10, 13),
 (1, 6, 10, 13, 14),
 (1, 6, 11, 13, 14),
 (3, 5, 6, 7, 9, 10),
 (3, 5, 6, 7, 9, 10, 12),
 (3, 5, 6, 9, 10),
 (3, 5, 6, 9, 10, 12),
 (3, 5, 6, 9, 10, 13),
 (3, 5, 6, 9, 10, 13, 14),
 (3, 5, 6, 9, 11, 14),
 (3, 5, 6, 9, 14),
 (3, 5, 6, 11, 13, 14),
 (3, 5, 7, 9, 14),
 (3, 5, 7, 10, 12),
 (3, 5, 7, 10, 12, 14),
 (3, 5, 9, 14),
 (3, 5, 

In [22]:
len(ineqT4)

50

In [23]:
n=5

Perms=list(permutations(range(n)))


d5=merge2(res5.get())


T5=[]
for key in d5.keys():
    T5.append(key[1])

ineqT5=[]
for t in T5:
    mode=True
    for p in Perms:
        pt=sorted(PermTuple(t,p))
        if isLectLess(t,pt):
            pass
        else:
            mode=False
            break
            
            
    if mode: 
        ineqT5.append(t)
              

In [24]:
len(ineqT5)

7443

In [25]:
n=2

Perms=list(permutations(range(n)))


d2=merge2(res2.get())


T2=[]
for key in d2.keys():
    T2.append(key[1])

ineqT2=[]
for t in T2:
    mode=True
    for p in Perms:
        pt=sorted(PermTuple(t,p))
        if isLectLess(t,pt):
            pass
        else:
            mode=False
            break
            
            
    if mode: 
        ineqT2.append(t)
              

In [26]:
len(ineqT2), len(ineqT3), len(ineqT4), len(ineqT5) #a(2), a(3), a(4), a(5)

(1, 4, 50, 7443)